In [1]:
# -*- coding: utf-8 -*-

## standard packages
import numpy as np
import os
import datetime

## Config Parster for initiation
import configparser

## HTTP scraping packages
from lxml import html
import requests

#import httplib2
import urllib

from bs4 import BeautifulSoup, SoupStrainer
import json

## Database
import sqlite3

from importlib import reload


In [2]:
os.getcwd()

'/media/florian/Volume/GitHub_Projects/UsedCarStats'

In [3]:
import src.utils as utils
reload(utils)

<module 'src.utils' from '/media/florian/Volume/GitHub_Projects/UsedCarStats/src/utils.py'>

First, let's get the specifics and urls saved in the config file

In [4]:
pagesConfig = configparser.ConfigParser()
pagesConfig.read("config/siteSpecifics.ini")

for section in pagesConfig.sections():
    print(section)
    #for option in pagesConfig.options(section):
        #print(option, pagesConfig.get(section, option))
    #    print (section, option)

WebPages
additionalAdInfo
dbFilepath
dbInit


In [5]:
searchPage = pagesConfig.get('WebPages', 'mobile.de')
print(searchPage)

http://suchen.mobile.de/fahrzeuge/search.html?damageUnrepaired=NO_DAMAGE_UNREPAIRED&scopeId=C&isSearchRequest=true&sortOption.sortBy=creationTime&sortOption.sortOrder=DESCENDING&pageNumber=


In [6]:
idPage = pagesConfig.get('WebPages', 'mobile.de_ID')
print(idPage)

http://suchen.mobile.de/fahrzeuge/details.html?id=


connect to database

In [8]:
reload(utils)
#database.close()
database = utils.DataBase(os.getcwd() + '/src/DataBase/carStats.db')
database.connect()

In [9]:
#database.deleteAllContents('car')
database.getTableNames()

[('car',)]

In [10]:
database.save()

initialize scraping object

In [11]:
scraping = utils.htmlScraping(searchPage, idPage, database)

set scraping limits and stepsize

In [12]:
scraping.setPriceStepList(0, 50000, 200)
priceStepList = scraping.getPriceStepList()
len(priceStepList)

251

In [99]:
250*10/60.

41.666666666666664

run scraping

In [13]:
scraping.autoScraping()

50 / 50, current price range: 0 - 200 Euros
 (70, 4)
50 / 50, current price range: 200 - 400 Euros
 (311, 4)
50 / 50, current price range: 400 - 600 Euros
 (956, 4)
Error during getting Information from Page at: 18
249121154
Expecting value: line 1 column 2 (char 1)
Error during translating/writing Information to DB at: 18
249121154
'NoneType' object has no attribute 'items'
Error during getting Information from Page at: 494
249058940
Expecting value: line 1 column 2 (char 1)
Error during translating/writing Information to DB at: 494
249058940
'NoneType' object has no attribute 'items'
Error during getting Information from Page at: 933
249000584
Expecting value: line 1 column 2 (char 1)
Error during translating/writing Information to DB at: 933
249000584
'NoneType' object has no attribute 'items'
50 / 50, current price range: 600 - 800 Euros
 (848, 4)
Error during getting Information from Page at: 612
249058863
Expecting value: line 1 column 2 (char 1)
Error during translating/writing 

KeyboardInterrupt: 

In [ ]:
database.save()
database.close()

In [97]:
allData = database.execute("""SELECT * from car""")
len(allData)

2540

In [65]:
#allData = np.array(allData)
allData[1900]

('1.119 cm³',
 'Fiesta Classic',
 'No Information',
 None,
 'No Information',
 'PETROL',
 'Ford',
 '15.08.2017',
 'No Information',
 'bottrop',
 'Verkaufe Ford Fiesta, das Auto läuft super und Reifen und Bremsen sind auch inordnung die Innenausstatung ist für ihr Alter auch  sehr gepflegt. ',
 3,
 'Stoff, Andere',
 200.0,
 '9000',
 '2017-08-15',
 'Car',
 'False',
 'Benzin',
 'Fiesta',
 'Ford Fiesta',
 '11/2017',
 '03/1996',
 'EUR',
 'No Information',
 None,
 248931485,
 'No Information',
 'USED',
 'FSBO',
 'No Information',
 None,
 'Kleinwagen',
 '19',
 '160.113 km',
 'No Information',
 None,
 5,
 'No Information',
 '4 (Grün)',
 46238,
 1996,
 '37 kW (50 PS)',
 '2/3',
 'DE',
 'Schaltgetriebe')

In [66]:
cursor = database.getCursor()
cursor.description

(('cubiccapacity', None, None, None, None, None, None),
 ('adspecificsmodeldescription', None, None, None, None, None, None),
 ('manufacturercolorname', None, None, None, None, None, None),
 ('featurevariants', None, None, None, None, None, None),
 ('consumption', None, None, None, None, None, None),
 ('adspecificsfuel', None, None, None, None, None, None),
 ('adspecificsmake', None, None, None, None, None, None),
 ('firstseen', None, None, None, None, None, None),
 ('climatisation', None, None, None, None, None, None),
 ('dealercity', None, None, None, None, None, None),
 ('description', None, None, None, None, None, None),
 ('adfirstregmonth', None, None, None, None, None, None),
 ('interior', None, None, None, None, None, None),
 ('adprice', None, None, None, None, None, None),
 ('adspecificsmakeid', None, None, None, None, None, None),
 ('lastseen', None, None, None, None, None, None),
 ('adsegment', None, None, None, None, None, None),
 ('isdealer', None, None, None, None, None, N

In [69]:
database.save()
#database.close()

In [70]:
dbDataIDs = np.array(database.execute("""SELECT adID from car""")) # ToDo: make a variable out of the name of the DB
dbDataIDs

array([[138831035],
       [195826833],
       [202907657],
       ..., 
       [248928887],
       [248931485],
       [248932279]])

In [73]:
pagesConfig = configparser.ConfigParser()
pagesConfig.read("config/siteSpecifics.ini")
attributes = dict()
for option in pagesConfig.options('dbInit'):
    attributes[option] = pagesConfig.get('dbInit', option)
attributes.keys()

dict_keys(['lastseen', 'firstregistration', 'emission', 'transmission', 'cubiccapacity', 'category', 'features', 'adspecificsmodeldescription', 'manufacturercolorname', 'adid', 'doorcount', 'airbag', 'color', 'numseats', 'numberofpreviousowners', 'description', 'featurevariants', 'dealerzip', 'interior', 'adprice', 'sellertype', 'fuel', 'adspecificsmodelid', 'firstseen', 'mileage', 'dealercountry', 'adcondition', 'adspecificsmake', 'adspecificsmodel', 'parkassist', 'adspecificshsn', 'hu', 'isdealer', 'consumption', 'adspecificsmakemodel', 'emissionssticker', 'adsegment', 'adspecificsfuel', 'power', 'countryversion', 'adfirstregmonth', 'adcurrency', 'adspecificsmakeid', 'climatisation', 'adfirstregyear', 'dealercity'])

### useful Database commands

In [ ]:
#connection = sqlite3.connect('src/dataBase/carStats.db')
#cursor = connection.cursor()

# delete 
#cursor.execute("""DROP TABLE car;""")

#cursor.execute(sql_create_command) -> create a proper 'create' command first
#cursor.description

# never forget this, if you want the changes to be saved:
#connection.commit()

#connection.close()

## progressBar tests

In [100]:
import sys, time
reload(utils)
pbar = utils.ProgressBar(10, 5)

for i in range(10):
    #print ('\r', i)
    pbar.update(i+1)
    time.sleep(.1)



10 / 10